<a href="https://colab.research.google.com/github/MahmoudKhaledd/Data-Modeling-with-Postgres/blob/main/survey_cleaning_combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guidelines:
* Upload qualitrics / inc-query data, main task, and call task data files to the 'content' folder on the left, by clicking the three dots and choosing 'upload file'

* Enter the survey type ('qual' or 'inc')
* Enter filenames as they are without the ".xlsx"
* Enter column names between single quotations seperated by a comma, and don't remove the brackets [ ]

* **brands_question** > Enter exact column name if all brands are in the same cell, or enter the question number, for example Q17, and it will scan the rest (Q17_1, Q17_2, etc) if each brand in a different cell

* **text_questions** >
   * **Qualtrics**: Enter column name complete, for example: "Q17_1"
   * **Inc-Query**: Just enter the column number, for example "Q45", script will scan the rest

* **straighlining_questions** > Enter column name without the "_" and it will scan the rest.





______________________________________________________________________


# Risk Score Calculations:
* **Duration < min** = +100
* **Duration > max** = +30
* **Straightlining** = +40
* **Text Duplication** = +100
* **Risk Rating**  "High" = +20
* **User IP Countries** > 1 = +50
* **Shared PayPal Accounts** > 1 = +100
* **Honesty Rating** < 10 AND (Lifetime Accepted Count + Lifetime DisqualificationCount > 50) = +10
* **User VPN Usage** > 50 = +100



______________________________________________________________________
# Notes

* In addition to the "Risk Score Calculations", the script dies the following:
  * Combines the inc-query / uqaltrics data with the pulse main and call tasks into one excel file.
  * Highlights the "good" users in green and the "bad" users in red.
  * Highlights if the "User Honesty Rating" is less than 51
  * Highlights the straightlined answers.
  * Highlights the duplicated answers.
  * If gc=0: (**Qualtrics Only**) 
    * Highlights the "Risk Score" in orange and label it as "rejected by screeners"
    * Action to Client will be "Rejected by Script"
    * Reason will be "gc=0".
  * If duration is less than the mininmum threhold entered by user:
    * "Action to client" will be "Rejected by Script"
    * "Reason" will be "Speeder" 
* Adds "Comments" column for all the violations present in the survey.


In [5]:
########################################################### EDIT HERE ##################################################################

# Enter 'inc' or 'qual'
survey_type = 'qual' 

main_task_filename = "MainSSG" # Enter without the ".xlsx" or ".csv"
call_task_filename = "CallSSG" # Enter without the ".xlsx" or ".csv"
qual_or_inc_filename = "QualSSG" # Enter without the ".xlsx" or ".csv"

# Enter the minimum and maximum duration threholds in minutes
min_duration_allowed_in_mins = 7
max_duration_allowed_in_mins = 1000
# Enter column name complete if all brands are in the same cell
# or enter column name without the prefix after the "_" if brands are in
# different cells|
brands_questions = ['Q17']
# Enter column name complete
text_questions = ['Q16','Q27','Q28']
# Enter column name without the prefix after the "_"
straightlineing_questions = []
# Enter column name complete
outliers_questions = []

# DO NOT EDIT BELOW
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################
#######################################################################################################################################

# changes made on 22/09/2022 by Carlos:
# changed the text duplicates section to make the runs more efficient, and it also spots text similarities instead of only duplicates
#######################################################################################################################################

import pandas as pd
import re
from datetime import datetime
import pandas as pd
import numpy as np
import re
import math
import warnings
warnings.filterwarnings("ignore")
from openpyxl import load_workbook
!pip install -U sentence_transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


def clean_headers(s):
  #Extracts question numbers from headers, for example: Q1, Q19, etc.
  try:
    r = re.findall('Q[0-9]+', s)[0]
    return r  
  except:
    return s


def rename(l):
  """
  Renames the duplicate question names so:
  ['Q1', 'Q1', 'Q1'] -> ['Q1_1', 'Q1_2', 'Q1_3']
  """
  seen = {}
  total = {}
  for x in l:
      if x not in total:
        total[x] = l.count(x)
  for x in l:
    if x in seen:
        seen[x] += 1
        yield "%s_%d" % (x, seen[x])
    elif x not in seen and total[x] > 1:
      seen[x] = 1
      yield "%s_1" % x
    else:
        yield x




def set_duration_criteria(x):
  if(x < 10):
    return "< 10% of average"
  if(x < 25):
    return "10%-25% of average"
  if(x < 50):
    return "25%-50% of average"
  if(x < 75):
    return "50%-75% of average"
  if(x < 90):
    return "75%-90% of average"
  else:
    return "> 90% of average"



def read_file(filename):
  # Reads the file whether it's an excel or csv
  try:
    df = pd.read_excel("{}.xlsx".format(filename))
  except:
    df = pd.read_csv("{}.csv".format(filename))
  return df

def clean_text(text):
  # Takes a str, returns it cleaned
  # remove numbers
  try:
    clean_text = ''.join([i for i in text if not i.isdigit()])
    # remove panctuation
    clean_text = re.sub(r'[^\w\s]','',clean_text)
    # remove spaces at the end, and converts to lowercase
    return clean_text.strip().lower()
  except:
    None

def straightline(MyList):
  """ 
  Takes a list, returns how many times the most frequent element appears
  eg. [1,2,3,4,4] > 2
    [1,2,3,4] > 0 (no most frequent element)
  """
  res = {}
  for i in MyList:
      res[i] = MyList.count(i)
  result = max(list(res.values()))
  if result == 1:
    result = 0     
  return(result)



def country_check(row):
  """
  Takes a df row, checks if Country equals or is in User IP Countries,
  if so, returns GOOD, else BAD
  """
  try:
    if row["country"] in row["User IP Countries"]:
      return "GOOD"
    else:
      return "BAD"
  except:
    None


def change_type(x):
    # Changes the type to float. If NULL, do nothing
    try:
        return float(x)
    except ValueError:
        return None


def move_column(df, pos, col_name):
  """ 
  Takes in the dataframe, position we want to move the column to,
  and column name, and moves it to this position
  """
  cols = list(df)
  cols.insert(pos, cols.pop(cols.index('{}'.format(col_name))))
  df = df.loc[:, cols]
  return df

def color_cell(idx, col, color):
  """ 
  Takes the index, column name, and the required color to color
  a specefic cell
  """
  color = 'background-color: {}'.format(color)
  df1.loc[idx, col] = color


def return_df1(x):  
  """ 
  This is required for df.style.apply as it takes in a function that
  returns the styler dataframe in order to style the main one:
  df.style.apply(return_df1)
  """
  return df1


# Read files into dataframes
df_main = read_file(main_task_filename)
df_call = read_file(call_task_filename)
df_ = read_file(qual_or_inc_filename)

if survey_type == 'inc':
  # Adjust columns and headers format
  df_["Duration (in seconds)"] = df_["End time (GMT)"] - df_["Start time (GMT)"]
  df_["Duration (in seconds)"] = df_["Duration (in seconds)"].dt.total_seconds()
  questions_inc = df_.columns.to_list()
  a_dict = pd.Series(questions_inc, index = df_.columns).to_dict()
  new_df = pd.DataFrame(a_dict, index = [0])
  df_ = new_df.append(df_).reset_index(drop = True)
  headers = list(df_.columns)
  headers_cleaned = list(map(clean_headers, headers))
  headers_cleaned = list(rename(headers_cleaned))
  df_.columns = headers_cleaned

# Append main and call task dataframes
df_tasks = df_main.append(df_call)


# The list of the columns needed from call and main task to be kept.
cols = ["Response Id", "Task Id", "Task Type", "User Id", "User Honesty Rating",
        "Risk Rating", "User Acquisition Source", "User Acquisition Campaign",
        "state", "Comment", "Status Reason", "Previous Comment", "country",
        "User IP Countries", "industry", "User Role", "Seniority",
        "Shared IP Users", "Shared PayPal Account", "User Distinct IP Count", 
        "User IP Different From Registration Country", "User VPN Usage Count", 
        "User Tor Usage Count", "Task Accepted Count", "Task Declined Count", 
        "Task Disqualification Count", "Failed Trick Questions", 
        "Lifetime Accepted Count", "Lifetime Declined Count", "Task Name", 
        "Lifetime Disqualification Count", "Lifetime Failed Trick Questions"]

users_data = [['0b9dfc1a-e5ed-44e9-b789-3a3d5c2a0bc2', 'James', 'Good'], \
              ['d9ec5b93-c2ae-454a-be62-cf0a69f08e71', 'Jersham', 'Good'], \
              ['40bdf3a8-e067-49bf-bccd-76a54625d11f', 'leonard mbrice', 'Good'], \
              ['6d335372-1bff-473b-97cb-3bcf10772272', 'ruslan nikolayenko', 'Good'], \
              ['f45f7d5b-92e1-41bb-86e1-66209a0d6eea', 'paniz', 'Good'], \
              ['5d50cb51-95e8-4fbe-82ba-6bad4b88f3de', 'Loreno', 'Good'], \
              ['d03dde68-0e5d-43ec-a1a6-4885838e571b', 'Herve', 'Good'], \
              ['18b7d0eb-483b-4c1d-8600-86e4bc8c190a', 'Gerti', 'Good'], \
              ['fcc812bd-2a2d-42a6-aed9-d8fc92d88fa7', 'Louis', 'Good'], \
              ['8664b72b-4328-4633-a09b-ec35a7227911', 'Primael Tindano', 'Good'], \
              ['2f428603-d574-4421-8cf3-2d989827b0ef', 'Kamela', 'Good'], \
              ['c7de7441-cf3c-48d0-8763-c5515e7c9997', 'Ledina', 'Good'], \
              ['3bc4df0f-74dc-45c2-9aac-cee40d0bd51c', 'cedric', 'Good'], \
              ['cc599d45-1ce6-4a09-9e60-0439083af294', 'giada', 'Good'], \
              ['a5989e95-5887-456a-9e59-6bdea3fe11f7', 'Elina', 'Good'], \
              ['6af15730-4a24-4386-80c5-82b0317b3c4d', 'Guira', 'Good'], \
              ['05659cf3-20f1-4454-8a42-2b7f03011d58', 'Leonard', 'Good'], \
              ['90f08520-1fd5-4535-8cf4-631530377e4f', 'Junior', 'Good'], \
              ['a62a06e9-df64-4db6-bdc5-ebfe35655d9e', 'Frank ', 'Good'], \
              ['34a19a6c-1b0f-4fd2-88af-2e5a90789583', 'Alla', 'Good'], \
              ['44ab9b66-7c86-4ac9-bbe6-99e6d5864e4a', 'Alla', 'Good'], \
              ['d6f90cab-a4c7-4275-9b44-c976dec3dbe8', 'Eris', 'Good'], \
              ['1cb60d01-5f11-42ad-a989-5973ad00f7e5', 'Max', 'Good'], \
              ['447907bf-f4d2-4e70-917e-4b67256988dc', 'Mark Alberto', 'Bad'], \
              ['6deeb122-487d-435a-9ecf-7b455aac388f', 'Jennifer Endres', 'Bad'], \
              ['c93f951e-4eb5-4629-b783-ffecb36fd735', 'saadatu yakubu', 'Bad'], \
              ['d5f77216-614c-43cb-ae66-cfce326957e9', 'eugene Paradero', 'Bad'], \
              ['4fa9555f-62fb-4d4e-abb4-2f4130dfb88d', 'Rutchi Chiong', 'Bad'], \
              ['527d7951-6e58-4434-bbf1-500f4a3f18a5', 'Vesna Djordjevic', 'Bad'], \
              ['601121d0-6150-41a7-a6ea-41fcd58ddb3b', 'Alessandro Marchi', 'Bad'], \
              ['e73520c6-1b84-49c7-a1f7-abe8d2758391', 'Lara M', 'Bad'], \
              ['47ff9ce0-2e49-42d9-b454-f3b24cc91039', 'Rose Fiona', 'Bad'], \
              ['2f1f890e-abd7-4a36-bf13-cd01dca8055f', 'Egir Cela', 'Bad'], \
              ['695583f3-654b-4e45-8ee4-d1f7b54627fd', 'Julia Bracco', 'Bad'], \
              ['581134d4-f88c-47c8-ad29-4701557b98eb', 'Fatima', 'Bad'], \
              ['b58a3261-13dd-47d0-bad6-7445fe470b05', 'Fatima', 'Bad'], \
              ['ee0275a9-665c-43e5-a14d-e50424c5de87', 'Fatima', 'Bad'], \
              ['37b6c2f7-4429-4326-93c4-c04a792e13c9', 'Philopatir', 'Bad'], \
              ['dcd37616-28b5-4852-a16d-44dcc7ae74af', 'Philopatir', 'Bad'], \
              ['bdbd6768-5e19-4b2b-bf7a-465a724c86e9', 'Fatimas friend', 'Bad']] 

# Create the users dataframe

df_users = pd.DataFrame(users_data, columns=['user_id', 'user_name', 'user_status'])


# Keeping an original list of the cols before we modify cols 
pulse_cols = cols

# Remove all columns from df_tasks but only keep te pulse cols
df_tasks = df_tasks[cols]


# Perform the country check
df_tasks["Country Check"] = \
df_tasks.apply(lambda row: country_check(row), axis=1)

# Move the Country Check column to the required position
df_tasks = move_column(df_tasks, 13, "Country Check")

if survey_type == 'inc':
  # Merge df tasks and df qualtrics on response ID as a new df
  df = df_tasks.merge(df_, how = "right", \
                    right_on = "Respondent", left_on = "Response Id")
else:
  # Merge df tasks and df qualtrics on response ID as a new df
  df = df_tasks.merge(df_, how = "right", \
                    right_on = "responseId", left_on = "Response Id")
  # If "GC" is uppercase, make it lower case to be used further
  df.rename(columns={"GC":"gc"}, inplace = True)

  # Move gc column to the start of the file
  df = move_column(df, 0, 'gc')


# Change the type of the duration column into float
df["Duration (in seconds)"] = \
df['Duration (in seconds)'].apply(change_type).dropna()
# Converting the duration in seconds into minutes
df["Duration (in minutes)"] = df["Duration (in seconds)"] / 60

# Move the duration in minutes column to the required position
df = move_column(df, 33, "Duration (in minutes)")

# Create three empty columns
df["Pulse Action"] = ""
df["Action to Client"] = ""
df["Reason"] = ""

# Move columns around to their required position
df = move_column(df, 34, "Reason")
df = move_column(df, 34, "Action to Client")
df = move_column(df, 34, "Pulse Action")
df = move_column(df, 15, "Country Check")
df = move_column(df, 4, "Task Name")
# Merging with users df
df = df.merge(df_users, how = 'left', left_on = 'User Id', right_on = 'user_id').drop(columns = ["user_id"])

# Moving columsn around
df = move_column(df, 5, 'user_status')
df = move_column(df, 5, 'user_name')

# Reset index of the main df
df = df.reset_index().drop(columns = ["index"])

# Extracting the question row now as it'll mess up our analysis later
questions_row = df.iloc[0]
# Initialzie the styler dataframe df1, to be used to color cells
df1 = pd.DataFrame('', index=df.index, columns=df.columns)
# Drop the question row
df = df.drop(0, axis = 0)





# Making a new df with no duration outliers to be used to calcualte the avg
q_low = df["Duration (in seconds)"].quantile(0.01)
q_hi  = df["Duration (in seconds)"].quantile(0.99)
df_no_outliers = df[(df["Duration (in seconds)"] < q_hi) & \
                    (df["Duration (in seconds)"] > q_low)]

# Calculating the average duration of the survey, no outliers are taken into
# consideration
avg_duration = df_no_outliers["Duration (in seconds)"].mean()


# Calculating the percentage of average for each record
df["Duration_Percentage_of_Average ({} mins)".format(avg_duration/60)] = \
df["Duration (in seconds)"].apply(lambda x: x/avg_duration*100)

# Adding a column for the average duration criteria based on average duration
df["Duration_Criteria"] = df["Duration_Percentage_of_Average ({} mins)". \
                             format(avg_duration/60)].apply \
                             (lambda x: set_duration_criteria(x))

if survey_type == 'qual':
  # Getting a list of all the questions in the survey
  questions = df.columns[list(df.columns).index("UserLanguage")+1: \
                       list(df.columns).index("responseId")]
else:
  # Getting a list of all the questions in the survey
  r = re.compile("Q\d.*")
  questions = list(filter(r.match, df.columns))

# Calculate the number of questions for each record
for idx in range(len(df)):
  if idx != 0:   
    cnt = 0
    for q in questions:
      if pd.isnull(df.loc[idx, q]) == False:
        cnt = cnt + 1
    df.at[idx, "Number of Questions Answered"] = cnt

# Calculating the number of brands in a questions
answers = []

# If there are any brands questions
if (len(brands_questions) > 0):
# Loop over the questions in brands questions
  for q in brands_questions:
    
    # If the question exists in the survey questions with the same name,
    # it means that all the brands are going to be in the same cell,
    # divided by a comma
    
    if (q in df.columns):
      for idx, row in df.iterrows():
        try:
          df.at[idx, "Number_of_brands_in_{}".format(q)] = \
            df.at[idx, q].count(",")+1
        except:
          None
    
    # If the questions does not exist with the same name, it means it contains
    # and underscore, e.g. Q1 in the brands questions and in the survey itself,
    # it is Q1_1, Q1_2, etc. This means the brands are going to be in different 
    # columns in this case  
    else:
      for idx, row1 in df.iterrows():
        for row2 in df:
          if (q in row2):
            if (isinstance(df.at[idx, "{}".format(row2)], str) or \
                isinstance(df.at[idx, "{}".format(row2)], int)):
              answers.append(df.at[idx, "{}".format(row2)])
        if(len(answers)>0):
          df.at[idx, "Number_of_brands_in_{}".format(q)] = str(len(answers))
          answers = []

if survey_type == 'inc':
  # Get all questions in text_questions
  # e.g if the user enters 'Q42' in the text questions, this would extract all of
  # 'Q42' such as 'Q42_1', 'Q42_2', 'Q42_3', etc
  new_text_questions = []
  for q in text_questions:
    for q2 in questions:
      if q in q2:
        new_text_questions.append(q2)
  text_questions = new_text_questions

### new text comparison
#create dictionaries to store the texts, lengths, and tensors
text_dict = {}
ten_dict = {}
len_cond = {}

#check if any text questions:
length_text = len(text_questions)
if length_text > 0:
  #for every question in the list:
  for k in range(length_text):
    q_in_q = text_questions[k]
    #create an entry in the different dictionaries for them
    text_dict[q_in_q] = []
    len_cond[q_in_q] = []
    #check each survey answer for the text question, to actually make sure it is text, and if it's empty replace the Nonetype object with N/A
    for i in list(df[q_in_q]):
      #pass through cleaner
      j = clean_text(i)
      #check if empty
      if j is None:
        #append 0 length and N/A string
        text_dict[q_in_q].append("N/A")
        len_cond[q_in_q].append(0)
      #otherwise append it's length and the clean text
      else:
        txt_len = len(j.split())
        text_dict[q_in_q].append(j)
        if txt_len >= 4:
          len_cond[q_in_q].append(1)
        else:
          len_cond[q_in_q].append(0)
    
    #encode through model the list of strings
    qq = text_dict[q_in_q]
    emb_qq = model.encode(qq, convert_to_tensor=True)
    ten_dict[q_in_q] = emb_qq

#create dictionary where we'll append the cosine scores 
cos_dict = {}
coscount = 0
whilecount = 0
lenmat_dict = {}
#if there are text questions:
if length_text > 0:
  #while there are elements to compare against:
  while whilecount <= (length_text - 1):
    #set base question
    base_q = text_questions[whilecount]
    #compare through all remaining elements in the list of text questions
    while coscount <= (length_text - 1):
      #check cosine similarities
      comp_q = text_questions[coscount]
      key = [base_q, comp_q]
      cos_score = util.cos_sim(ten_dict[key[0]],ten_dict[key[1]])
      cos_dict[tuple(key)] = cos_score
      coscount += 1
      #creating the conditional length matrix
      a = np.array(len_cond[base_q])
      b = np.array(len_cond[comp_q])
      ab = np.outer(a,b)
      lenmat_dict[tuple(key)] = ab

    #add one to the whilecount, ie select next element in the list of text_questions as base_q, and set the first element to be compared to be itself
    whilecount += 1
    coscount = whilecount

#we are now going to select only elements having higher than a certain score, ie text that a similar, and we are going to include the length check
#for every key pair of questions in the cosine dictionary
for key in cos_dict:
  #get values
  mat = cos_dict[key].numpy()
  len_mat = lenmat_dict[key]
  #get elements higher than a certain threshold
  cos_cond = (mat >= 0.95).astype(int)
  #multiply both matrices element wise to satisfy both conditions
  condition = np.multiply(len_mat,cos_cond)
  #get indices of the elements
  higher = np.transpose(condition.nonzero())
  #check if we are comparing a question with itself, if so don't pay attention to text answers being compared with themselves, since score will be 1
  if key[0] == key[1]:
    for i in higher:
      prim = i[0]
      sec = i[1]
      if (prim != sec):
        color_cell(int(prim)+1, key[0], "yellow")
        df.at[int(prim)+1, "Duplication_Within_Survey in {}".format(key[0])] = "Duplication from record number {} in {}".format(int(sec)+3, key[1])
  else:
    for i in higher:
      prim = i[0]
      sec = i[1]
      color_cell(int(prim)+1, key[0], "yellow") 
      df.at[int(prim)+1, "Duplication_Within_Survey in {}".format(key[0])] = "Duplication from record number {} in {}".format(int(sec)+3, key[1])
      
      

# Checking the straighlining criteria
answers = []
if (len(straightlineing_questions) > 0):
  for idx, row1 in df.iterrows():
    for q in straightlineing_questions:
      # Making sure it is not a text question
      if ("TEXT" not in q):
        for row2 in df:
          if (q in row2):
            # Making sure the answer is either a str or an int
            if (isinstance(df.at[idx, "{}".format(row2)], str) or \
                isinstance(df.at[idx, "{}".format(row2)], int)):
              # Append the answer to the answers list
              answers.append(df.at[idx, "{}".format(row2)])
        # If the list is not empty
        if(len(answers)>0):
          # Calcualte the straightlining percentage 
          df.at[idx, "Straightlineing_in_{}".format(q)] = \
          str(math.floor(straightline(answers) / len(answers) * 100)) + "%"
          # Checking if the straighlining percentage is more than 90%
          if math.floor(straightline(answers) / len(answers) * 100) >= 90:
            for row2 in df:
              # Making sure question name does not inculde Hq or HQ, and then
              # highlight all the cells straighlines
              if (q in row2 and "Hq" not in row2 and "HQ" not in row2):
                color_cell(idx, row2, "#9bdfd4")
        answers = []

# Checking the outliers criteria
if(len(outliers_questions) > 0):
  for q in outliers_questions:
    if q in df.columns:
      q_low = df["{}".format(q)].quantile(0.01)
      q_hi  = df["{}".format(q)].quantile(0.99)
      for idx, row in enumerate(questions):
        idx = idx + 1
        if df.at[idx, q] > q_hi or df.at[idx, q] < q_low:
          df.at[idx, "Outliers_in_{}".format(q)] = "True"

# Checking the Duration Strike based on the threshold the user entered
for idx, row in df.iterrows():
  if df.at[idx, "Duration (in minutes)"] < min_duration_allowed_in_mins:
    df.at[idx, "Duration_Strike"] = 1
  elif df.at[idx, "Duration (in minutes)"] > max_duration_allowed_in_mins:
    df.at[idx, "Duration_Strike"] = 2
  else:
    df.at[idx, "Duration_Strike"] = 0


"""
Aggregating all our past findings in new columns:
Straightlined_questions > Number of questions a user straighlined
Duplicated_Questions > Number of questions a user duplicated
Duplication_Percentage > The percentage of the questions a user puplicated
Outliers_Questions > Number of questions a user entered an outlier value
"""

if (len(straightlineing_questions) > 0):
  straightlines = 0
  for idx, row in df.iterrows():
    for q in df.columns:
      if "Straightlineing" in q:
        try:
          if (int(df.at[idx, q].replace("%", "")) >= 90):
            straightlines = straightlines + 1
        except:
          None
    df.at[idx, "Straightlined_Questions"] = straightlines
    straightlines = 0


if (len(text_questions) > 0):
  c = 0
  for idx in range(len(df)):
    idx = idx + 1
    for q in df.columns:
      if "Duplication" in q:
        if isinstance(df.at[idx, q], str):
          c = c+1
    df.at[idx, "Duplicated_Questions"] = c
    c = 0


# Calculating the Duplication percentages of the duplicated questions to
# all of the text questions
if "Duplication_Questions" in df.columns:
  df["Duplication_Percentage"] = df["Duplication_Questions"] / \
                                 len(text_questions)*100

if (len(outliers_questions) > 0):
  c = 0
  for idx in range(len(df)):
    idx = idx + 1
    for q in df.columns:
      if "Outliers" in q:
        if isinstance(df.at[idx, q], str) and df.at[idx, q] == "True":
          c = c+1
    df.at[idx, "Outliers_Questions"] = c
    c = 0

# Calculating the Final Score based on all of the above criterias
for idx in range(len(df)+1):
  if(idx != 0):
    cnt = 0
    comments = []
    for q in df.columns:
      if q in ['Straightlined_Questions', \
               'Duplicated_Questions', 'Outliers_Questions']:
        if isinstance(df.at[idx, q], float):
          cnt = cnt + df.at[idx, q]
          if (df.at[idx, q] > 0):
            comments.append(q)
      elif q == "Duration_Strike":
        if df.at[idx, q] == 1:
          cnt = cnt + 1
          comments.append(q + ' +100')
        elif df.at[idx, q] == 2:
          cnt = cnt + 1
          comments.append(q + ' +30')
      elif q == "Risk Rating":
        if df.at[idx, q] == 'HIGH':
          cnt = cnt + 1
          comments.append(q)
      elif q == "User IP Countries":
        if not pd.isnull(df.at[idx, q]) and df.at[idx, q].count(',') >= 1:
          cnt = cnt + 1
          comments.append(q)
      #elif q == "Shared IP Users":
        #if not pd.isnull(df.at[idx, q]) and df.at[idx, q].count(',') >= 3:
          #cnt = cnt + 1
          #comments.append(q)
      elif q == "Shared PayPal Account":
        if not pd.isnull(df.at[idx, q]) and df.at[idx, q].count(',') >= 1:
          cnt = cnt + 1
          comments.append(q)
      #elif q == "Shared PayPal Account":
        #if not df.at[idx, q] > 1:
          #cnt = cnt + 1
          #comments.append(q)
      elif q == "Honesty rating":
        if not pd.isnull(df.at[idx, q]) and \
          df.at[idx, q] < 10 and \
          (df.at[idx, "Lifetime Accepted Count"] + \
          df.at[idx, "Lifetime Disqualification Count"]) < 10: 
          cnt = cnt + 1
          comments.append(q)
      elif q == "User VPN Usage Count":
        #if not pd.isnull(df.at[idx, q]):
        if (df.at[idx, q] > 50):
          cnt = cnt + 1
          comments.append(q)

    df.at[idx, "Final_Score"] = cnt
    df.at[idx, "Comments"] = str(comments)

# calculating the Risk Score

for idx in range(len(df)+1):
  if(idx != 0):
    cnt = 0
    for q in df.columns:
      if q == 'Duration (in minutes)':
        if df.at[idx, q] < min_duration_allowed_in_mins:
          cnt = cnt + 100
        elif df.at[idx, q] > max_duration_allowed_in_mins:
          cnt = cnt + 30
      elif q == 'Straightlined_Questions':
        if df.at[idx, q] >= 1:
          cnt = cnt + 40
      elif q == 'Duplicated_Questions':
        if df.at[idx, q] >= 1:
          cnt = cnt + 100
      elif q == 'Risk Rating':
        if df.at[idx, q] == 'HIGH':
          cnt = cnt + 20
      elif q == 'User IP Countries':
        if not pd.isnull(df.at[idx, q]) and df.at[idx, q].count(',') >= 1:
          cnt = cnt + 50
      #elif q == 'Shared IP Users':
        #if not pd.isnull(df.at[idx, q]) and df.at[idx, q].count(',') >= 3:
          #cnt = cnt + 20
      elif q == 'Shared PayPal Account':
        if not pd.isnull(df.at[idx, q]):
          accs = df.at[idx, q].count(',')+1
          if accs > 1:
            cnt = cnt + 100
      #elif q == 'Shared PayPal Account':
        #if not df.at[idx, q] > 1:      
          #cnt = cnt + 100 
      elif q == "Honesty rating":
        if not pd.isnull(df.at[idx, q]) and \
          df.at[idx, q] < 10 and \
          (df.at[idx, "Lifetime Accepted Count"] + \
          df.at[idx, "Lifetime Disqualification Count"]) < 10:
          cnt = cnt + 10
      elif q == "User VPN Usage Count":
        #if not pd.isnull(df.at[idx, q]):
          #if (df.at[idx, q] >= 10 and df.at[idx, q] <= 20):
            #cnt = cnt + 20
          #if (df.at[idx, q] > 20 and df.at[idx, q] <= 51):
            #cnt = cnt + 30
        if (df.at[idx, q] >= 50):
          cnt = cnt + 100
    # Risk Score cannot exceed 100%
    if cnt > 100:
      cnt = 100
    df.at[idx, "Risk_Score"] = str(cnt)+'%'
    if cnt >= 100:
      color_cell(idx, "Risk_Score", "red")



# Highlight the records disqualified by screeners aka 'gc' = 0 
for idx in range(len(df)+1):
  if(idx != 0):
    if df.at[idx, "gc"] == '0':
      df.at[idx, "Risk_Score"] = "Disqualified by Screeners"
      color_cell(idx, "Risk_Score", "orange")

# Highlighting records with Honesty Rating less than 51
for idx in range(len(df)+1):
  if(idx != 0):
    try:
      if df.at[idx, "User Honesty Rating"] < 51:
        color_cell(idx, "User Honesty Rating", "yellow")
    except:
      None

# Highliting Good and Bad users
for idx in range(len(df)+1):
  if(idx != 0):
    try:
      if df.at[idx, "user_status"] == 'Good':
        color_cell(idx, "User Id", "green")
        color_cell(idx, "user_name", "green")
        color_cell(idx, "user_status", "green")
      elif df.at[idx, "user_status"] == 'Bad':
        color_cell(idx, "User Id", "red")
        color_cell(idx, "user_name", "red")
        color_cell(idx, "user_statsu", "red")
    except:
      None


# Highlight the columns headers accordingly
for col in df.columns:
  if col in ["Country Check", "Duration (in minutes)", "Pulse Action", \
             "Action to Client", "Reason"]:
    color_cell(0, col, "#FFCCCB")
  elif col in pulse_cols:
    color_cell(0, col, "#ADD8E6")
  else:
    color_cell(0, col, "#D3D3D3")
color_cell(0, "Response Id", "#ADD8E6")

# new update - Fill Action to Client & Reason in case of gc=0 or speeder
# gc = 0
for idx in range(len(df)+1):
  if(idx != 0):
    if df.at[idx, "gc"] == '0':
      df.at[idx, "Action to Client"] = "Rejected by Script"
      df.at[idx, "Reason"] = "gc=0"
# speeder
for idx in range(len(df)+1):
  if(idx != 0):
    if df.at[idx, "Duration (in minutes)"] < min_duration_allowed_in_mins:
      df.at[idx, "Action to Client"] = "Rejected by Script"
      df.at[idx, "Reason"] = "speeder"



df["Risk_Score_"] = df["Risk_Score"]
df["Comments_"] = df["Comments"]



# Append the question row we removed at the start to the dataframe again
df = pd.DataFrame(questions_row.to_dict(), index = [0]).append(df)

df = move_column(df, 39, "Risk_Score_")
df = move_column(df, 39, "Comments_")


# Applying the styling to the dataframe
df1 = df1.reindex(columns = df.columns)
# to avoid float error
df = df.fillna("")
df1 = df1.fillna("")
df = df.style.apply(return_df1, axis=None)

## new part to test
#df.to_excel("master_clean.xlsx", sheet_name = "master")
#FilePath = "master_clean.xlsx"
#ExcelWorkbook = load_workbook(FilePath)
#writer = pd.ExcelWriter(FilePath, engine = 'openpyxl')
#writer.book = ExcelWorkbook
#writer.save()

xlwriter = pd.ExcelWriter('master_clean.xlsx')
df.to_excel(xlwriter, sheet_name = 'master', index = False)
df_main.to_excel(xlwriter, sheet_name = 'main task', index = False)
df_call.to_excel(xlwriter, sheet_name = 'call task', index = False)
df_users.to_excel(xlwriter, sheet_name = 'users', index = False)

xlwriter.close()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
